<h1> Conditional Edge </h1> 

### Intro
Life isn't always a straight line. Sometimes we need to make choices.
In LangGraph, we use **Conditional Edges** to route execution based on the state.

### The Scenario
We want to build a calculator that receives two numbers and an operator.
- If operator is `+`, go to `adder` node.
- If operator is `-`, go to `subtractor` node.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

### State
Standard calculator state: two numbers, an operator, and a result.

In [ ]:
class AgentState(TypedDict): #state schema
    num1: int
    operator: str
    num2: int
    result: int

### The Workers
Two simple nodes. One adds, one subtracts.

In [ ]:
def adder(state: AgentState) -> AgentState:
    """This node adds the two numbers and stores the result in 'result'"""
    state["result"] = state["num1"] + state["num2"]
    return state

In [ ]:
def subtractor(state: AgentState) -> AgentState:
    """This node subtracts num2 from num1"""
    state["result"] = state["num1"] - state["num2"]
    return state

### The Router Logic
This isn't a node in the graph per se, but a function that returns the *name* of the next path. It looks at the state and says "go this way" or "go that way".

In [ ]:
def routerFunc(state: AgentState) -> str: # returns name of the edge
    """This is a function node which carries the conditional logic.""" 
    if(state["operator"] == "+"):
        return "addEdge"
    elif(state["operator"] == "-"):
        return "subEdge"

### Building the Graph with Conditions
1. Add worker nodes.
2. Add a `router` node (often just a pass-through using lambda).
3. Use `add_conditional_edges` to connect the router to the workers based on `routerFunc` output.

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("adder", adder)
graph.add_node("subtractor", subtractor)
graph.add_node("router", lambda x: x) # passthrough function - does nothing but route

graph.add_edge(START, "router")

graph.add_conditional_edges(
    "router",        # From this node
    routerFunc,      # Using this logic
    {
        # If Logic returns this : Go to this Node
        "addEdge": "adder", 
        "subEdge": "subtractor"
    }
)

graph.add_edge("adder", END)
graph.add_edge("subtractor", END)

app = graph.compile()

### Visualization
Notice how the router splits into two paths? That's your conditional edge at work.

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

### Test Drive
Let's try subtraction.

In [ ]:
example = app.invoke({"num1": 1, "num2": 3, "operator": "-"})
print(example) #prints the entire state 